原始数据：
https://osf.io/8f6n9 Vero_SCV2_1.fq0.zst
https://osf.io/8f6n9 Vero_SCV2_2.fq0.zst

解压：
zstd -d Vero_SCV2_1.fq0.zst && zstd -d Vero_SCV2_2.fq0.zst

去除低质量序列和接头：fastp v0.23.2
fastp -i Vero_SCV2_1.fq0 -I Vero_SCV2_2.fq0 -o SARS2.read1.clean.fq -O SARS2.read2.clean.fq -q 20 -u 30 -f 10

fastx_toolkit v0.0.14
下载绿猴全基因组序列：wget -O genome.fa.gz ftp://ftp.ensembl.org/pub/release-99/fasta/chlorocebus_sabaeus/dna/Chlorocebus_sabaeus.ChlSab1.1.dna_sm.toplevel.fa.gz
下载绿猴全基因组序列注释：wget -O genome.gff3.gz ftp://ftp.ensembl.org/pub/release-99/gff3/chlorocebus_sabaeus/Chlorocebus_sabaeus.ChlSab1.1.99.gff3.gz
合并基因组：(cat SARS-CoV-2.fa ont-refs.fa human-rDNA.fa; zcat genome.fa.gz) | fasta_formatter -w 72 | bgzip -@ 20 -c /dev/stdin > SARS2_link.genome.fa.gz
合并注释：(cat SARS-CoV-2.gff3 ont-refs.gff3 human-rDNA.gff3 ;  zcat genome.gff3.gz) | bgzip -@ 20 -c /dev/stdin > SARS2_link.genome.gff3.gz

#建索引：samtools v1.19
#samtools faidx SARS2_link.genome.fa.gz

解压基因组：
zcat SARS2_link.genome.fa.gz > SARS2_link.genome.fa

生成star索引：STAR v2.7.1a
mkdir -p SARS2_link.genome.star
STAR --runThreadN 50 --runMode genomeGenerate --genomeDir SARS2_link.genome.star --genomeFastaFiles SARS2_link.genome.fa

star比对：
STAR --runThreadN 50 --genomeDir reference/SARS2_link.genome.star --readFilesIn CRR197318_1.fq.gz CRR197318_2.fq.gz --readFilesCommand zcat --outFilterType BySJout --outFilterMultimapNmax 20 --alignSJoverhangMin 8 --alignSJDBoverhangMin 1 --outSJfilterOverhangMin 12 12 12 12 --outSJfilterCountUniqueMin 1 1 1 1 --outSJfilterCountTotalMin 1 1 1 1 --outSJfilterDistToOtherSJmin 0 0 0 0 --outFilterMismatchNmax 999 --outFilterMismatchNoverReadLmax 0.04 --scoreGapNoncan -4 --scoreGapATAC -4 --chimOutType Junctions WithinBAM HardClip --chimScoreJunctionNonGTAG 0 --alignSJstitchMismatchNmax -1 -1 -1 -1 --alignIntronMin 20 --alignIntronMax 1000000 --alignMatesGapMax 1000000 --chimSegmentMin 20 --outFileNamePrefix SARS2.genome.star --outSAMmultNmax 32 --outSAMtype BAM SortedByCoordinate  --outSAMunmapped Within KeepPairs --sjdbGTFfile references/SARS2_link.genome.gtf --quantMode GeneCounts

ln alignments/SARS2.genome.star/Aligned.sortedByCoord.out.bam alignments/SARS2.genome.bam

给bam文件建索引:
samtools index SARS2.genome.bam

提取比对上病毒的reads:
samtools view -@ 50 -b -o alignments/SARS2.virus.bam alignments/SARS2.genome.bam chrSCV
samtools view -c -f 1 -F 12 test.bam

提取cigar值中存在jump的reads:
samtools  view alignments/SARS2.virus.bam | awk 'BEGIN {{ OFS="\t"; }} {{ if ($6 ~ /N/) print $4, $6; }}' | bgzip -c -@ 4 /dev/stdin > jumpstats/SARS2.jump-cigars.txt.gz

统计jump的位置:
zcat jumpstats/SARS2.jump-cigars.txt.gz | python convert-cigars.py | sort -k1,2n | uniq -c | awk '{{ OFS="\t"; }} {{ print $2, $3, $1; }}' | bgzip -c /dev/stdin > jumpstats/SARS2.jumps.txt.gz

统计覆盖度:bedtools v
bedtools genomecov -ibam alignments/SARS2.virus.bam  -dz -split > tables/SARS2.virus.coverage.txt